In [53]:
import numpy as np
import pandas as pd

In [55]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credit = pd.read_csv('tmdb_5000_credits.csv')

In [56]:
df = movies.merge(credit,on='title')

In [57]:
df = df[['id','title','genres','keywords','overview','crew','cast']]

In [58]:
df.isnull().sum()

id          0
title       0
genres      0
keywords    0
overview    3
crew        0
cast        0
dtype: int64

In [59]:
df.dropna(inplace = True)

In [60]:
import ast
def convert(obj):
    a = []
    for i in ast.literal_eval(obj):
        a.append(i['name'])
    return a;
df['genres'] = df['genres'].apply(convert)
df['keywords'] = df['keywords'].apply(convert)

In [61]:
def cast_convert(obj):
    a = []
    j = 0
    for i in ast.literal_eval(obj):
        a.append(i['name'])
        j += 1
        if j == 3:
            return a
df['cast'] = df['cast'].apply(cast_convert)

In [62]:
def crew_convert(obj):
    a = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            a.append(i['name'])
            break
    return a
df['crew'] = df['crew'].apply(crew_convert)

In [63]:
df['overview'] = df['overview'].apply(lambda x:x.split())

In [64]:
df['genres'] = df['genres'].apply(lambda x:[i.replace(" ","") for i in x])
df['keywords'] = df['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [65]:
df['crew'] = df['crew'].apply(lambda x: [i.replace(" ","") for i in x] if isinstance(x, list) else [])
df['cast'] = df['cast'].apply(lambda x: [i.replace(" ","") for i in x] if isinstance(x, list) else [])

In [66]:
df['tags'] = df['overview']+df['genres']+df['keywords']+df['cast']+df['crew'] 

In [67]:
data = df[['id','title','tags']]

In [73]:
data['tags'] = data['tags'].apply(lambda x:" ".join(x))

C:\Users\DELL\AppData\Local\Temp\ipykernel_32968\39123791.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tags'] = data['tags'].apply(lambda x:" ".join(x))


In [74]:
import nltk

In [75]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stemmer(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
data['tags'] = data['tags'].apply(stemmer)

C:\Users\DELL\AppData\Local\Temp\ipykernel_32968\2788358111.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tags'] = data['tags'].apply(stemmer)


In [76]:
data

,id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just want to play hi guitar and ca...
4805,72766,Newlyweds,a newlyw couple' honeymoon is upend by the arr...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduc a dedic q..."
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...


In [78]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [79]:
vector = cv.fit_transform(data['tags']).toarray()

In [99]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [80]:
vector.shape

(4806, 5000)

In [81]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [82]:
from difflib import SequenceMatcher

def sentence_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

def recomend(movie):
    movie_idx = data[data['title'].str.lower() == movie.lower()]
    if not movie_idx.empty:
        movie_idx = movie_idx.index[0]
        distance = similarity[movie_idx] 
        movie_list = sorted(list(enumerate(distance)),reverse=True,key = lambda x: x[1])[0:6]
        for i in movie_list:
            print(data.iloc[i[0]].title);

    word_similarity = []        
    for i,df_movie in enumerate(data['title'].str.lower()):
        ratio = sentence_similarity(df_movie,movie.lower())
        if ratio >= 0.6:
            word_similarity.append(i)
    if len(word_similarity) == 0:
        print("not found")
    for i in word_similarity:
        print(data.iloc[i].title);
recomend('Jurassic')

Jurassic World
Jurassic Park III
Jurassic Park
Basic


In [ ]:
import pickle

In [ ]:
pickle.dump(data.to_dict(),open('movie_dist.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity_mtx.pkl','wb'))